In [1]:
import nibabel as nib  
import numpy as np
from pathlib import Path

In [2]:
base_dir = Path('/home/rusk/projects/FYP-KiTS/KiTS')

In [3]:
def trim_imaging_with_segmentation(imaging_file_path, segmentation_file_path, output_imaging_file_path, output_segmentation_file_path):
    # Load segmentation file
    segmentation_img = nib.load(segmentation_file_path)
    segmentation_data = segmentation_img.get_fdata()

    # Determine start and end frames based on segmentation mask
    start_frame = None
    end_frame = None
    for frame in range(segmentation_data.shape[-1]):
        if np.any(segmentation_data[..., frame]):
            start_frame = max(0, frame - 3) # Include 3 frames above the first mask
            break
    for frame in range(segmentation_data.shape[-1] - 1, -1, -1):
        if np.any(segmentation_data[..., frame]):
            end_frame = min(segmentation_data.shape[-1] - 1, frame + 3)  # Include 3 frames below the end mask
            break

    if start_frame is None or end_frame is None:
        print("No segmentation mask found.")
        return

    # Load imaging file
    imaging_img = nib.load(imaging_file_path)
    imaging_data = imaging_img.get_fdata()

    # Trim imaging data keeping 3 frames
    trimmed_imaging_data = imaging_data[..., start_frame:end_frame+1]

    # Trim segmentation data keeping 3 frames
    trimmed_segmentation_data = segmentation_data[..., start_frame:end_frame+1]

    # Save trimmed imaging data to a new file
    trimmed_imaging_img = nib.Nifti1Image(trimmed_imaging_data, imaging_img.affine)
    nib.save(trimmed_imaging_img, output_imaging_file_path)

    # Save trimmed segmentation data to a new file
    trimmed_segmentation_img = nib.Nifti1Image(trimmed_segmentation_data, segmentation_img.affine)
    nib.save(trimmed_segmentation_img, output_segmentation_file_path)

    print(f"Trimmed imaging data saved to {output_imaging_file_path}.")
    print(f"Trimmed segmentation data saved to {output_segmentation_file_path}.")

In [4]:
# Iterate through each case folder in KiTS
for case_folder in base_dir.glob('case_*'):
    case_name = case_folder.name
    # Process subfolders A and D
    for subfolder in ['A', 'D']:
        subfolder_path = case_folder / subfolder
        # Check if the subfolder exists
        if subfolder_path.exists():
            segmentation_file = subfolder_path / 'segmentation.nii.gz'  
            if segmentation_file.exists():
                print("find original files in folder:")
                print(subfolder_path)
                # modify voxel spacing and orientation in segmentation to imaging
                trim_imaging_with_segmentation(subfolder_path / "imaging.nii.gz", subfolder_path / "segmentation.nii.gz", subfolder_path / "imaging.nii.gz", subfolder_path / "segmentation.nii.gz")
            else:
                print("Canot find the segmentation.nii.gz in " + str(subfolder_path))

find original files in folder:
/home/rusk/projects/FYP-KiTS/KiTS/case_00015/A
Trimmed imaging data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/imaging.nii.gz.
Trimmed segmentation data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00015/A/segmentation.nii.gz.
find original files in folder:
/home/rusk/projects/FYP-KiTS/KiTS/case_00015/D
Trimmed imaging data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/imaging.nii.gz.
Trimmed segmentation data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00015/D/segmentation.nii.gz.
find original files in folder:
/home/rusk/projects/FYP-KiTS/KiTS/case_00014/A
Trimmed imaging data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00014/A/imaging.nii.gz.
Trimmed segmentation data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00014/A/segmentation.nii.gz.
find original files in folder:
/home/rusk/projects/FYP-KiTS/KiTS/case_00014/D
Trimmed imaging data saved to /home/rusk/projects/FYP-KiTS/KiTS/case_00014/D/imaging.nii.gz.
Trimmed